In [1]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (azure-mgmt-resource 20.0.0 (/Users/axelsirota/repos/deploying-managing-models-azure/.venv/lib/python3.7/site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1')).


In [2]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='sklearn_regression_model.pkl',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='0.24.1',             # Version of scikit-learn used to create the model.
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

Registering model sklearn_regression_model.pkl
Name: sklearn_regression_model.pkl
Version: 2


In [3]:
%%writefile "requirements_env.txt"

azureml-defaults
azureml-core
inference-schema[numpy-support]
numpy
scikit-learn==0.24.1
scipy
joblib


Overwriting requirements_env.txt


In [4]:
from azureml.core import Environment

environment=Environment.from_pip_requirements('my-sklearn-environment', file_path="requirements_env.txt")
environment.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20211029.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "my-sklearn-environment",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "cond

In [5]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoring_script.py', 
                                   source_directory='.',
                                   environment=environment)

In [6]:


from azureml.core.compute import AksCompute
from azureml.core.compute import ComputeTarget
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration(cluster_purpose="DevTest")

aks_name = 'my-aks-dev' 
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
# Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config) 
if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)



Found existing cluster, use it.


In [7]:
# deploying the model and create a new endpoint
from azureml.core.webservice import AksEndpoint
import string
import random
from azureml.core.compute import ComputeTarget

#select a created compute
compute = ComputeTarget(ws, 'my-aks-dev')
namespace_name="endpointnamespace"
# define the endpoint name
endpoint_name = f"myendpoint1{''.join(random.choice(string.ascii_lowercase) for _ in range(5))}"[:32]
# define the service name
print(endpoint_name)
version_name= "version1"

endpoint_deployment_config = AksEndpoint.deploy_configuration(tags = {'modelVersion':'firstversion', 'department':'finance'}, 
                                                              description = "my first version", namespace = namespace_name, 
                                                              version_name = version_name, traffic_percentile = 40, )

endpoint = Model.deploy(ws, endpoint_name, [model], inference_config, endpoint_deployment_config, compute)
endpoint.wait_for_deployment(True)

myendpoint1yeuzy
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-13 00:35:11-03:00 Creating Container Registry if not exists.
2021-12-13 00:35:11-03:00 Registering the environment.
2021-12-13 00:35:13-03:00 Building image for version1..
2021-12-13 00:46:29-03:00 Creating resources in AKS.
2021-12-13 00:46:29-03:00 Submitting deployment to compute.
2021-12-13 00:46:30-03:00 Checking the status of deployment version1..
2021-12-13 00:47:20-03:00 Checking the status of inference endpoint version1.
Succeeded
AKSENDPOINT service creation operation finished, operation "Succeeded"


In [8]:
version_name_add="version2" 

endpoint.create_version(version_name = version_name_add, inference_config=inference_config, models=[model], tags = {'modelVersion':'secondversion', 'department':'finance'}, 
                        description = "my second version", traffic_percentile = 10)
endpoint.wait_for_deployment(True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-13 00:50:32-03:00 Creating Container Registry if not exists.
2021-12-13 00:50:33-03:00 Registering the environment.
2021-12-13 00:50:34-03:00 Use the existing image for version1.
2021-12-13 00:50:36-03:00 Creating resources in AKS.
2021-12-13 00:50:37-03:00 Submitting deployment to compute.
2021-12-13 00:50:37-03:00 Checking the status of deployment version1..
2021-12-13 00:51:16-03:00 Checking the status of deployment version2..
2021-12-13 00:51:26-03:00 Checking the status of inference endpoint version1.
2021-12-13 00:51:27-03:00 Checking the status of inference endpoint version2.
Succeeded
AKSENDPOINT service creation operation finished, operation "Succeeded"


In [9]:
import json
test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})

test_sample_encoded = bytes(test_sample, encoding='utf8')
prediction = endpoint.run(input_data=test_sample_encoded)
print(prediction)

[11055.977245525679, 4503.079536107787]


In [10]:
endpoint.delete_version(version_name=version_name)
endpoint.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-13 00:55:00-03:00 Creating Container Registry if not exists.
2021-12-13 00:55:01-03:00 Use the existing image for version2.
2021-12-13 00:55:03-03:00 Checking the status of deployment version2..
2021-12-13 00:55:25-03:00 Checking the status of inference endpoint version2.
Succeeded
AKSENDPOINT service creation operation finished, operation "Succeeded"


In [11]:
prediction = endpoint.run(input_data=test_sample_encoded)
print(prediction)

[11055.977245525679, 4503.079536107787]


In [12]:
endpoint.delete()